# Ontology Loading

In [2]:
import owlready2
from owlready2 import get_ontology
import json
import re

In [47]:
# Set the path to the OWL/XML file
ontology_path = '/Users/faridahelmy/Downloads/OntologyNew.rdf'

# Load the ontology
onto = get_ontology(ontology_path).load()
print("Ontology loaded successfully!")

Ontology loaded successfully!


In [4]:
# Helper function to get a readable name
def get_readable_name(entity):
    return entity.label.first() if entity.label else entity.name

# List all classes with readable names
readable_class_names = [get_readable_name(cls) for cls in onto.classes()]
print("Classes:", readable_class_names)

# List all object properties with readable names
readable_object_property_names = [get_readable_name(prop) for prop in onto.object_properties()]
print("Object Properties:", readable_object_property_names)

# List all data properties with readable names
readable_data_property_names = [get_readable_name(prop) for prop in onto.data_properties()]
print("Data Properties:", readable_data_property_names)

Classes: ['Student', 'Course']
Object Properties: ['didNotTakeYet', 'droppedAttendance', 'failed', 'earlier', 'hasCompleted', 'hasNotCompleted', 'hasPrerequisite', 'invalidEnrollment', 'regularFailed', 'satisfiesPrerequisite', 'takes']
Data Properties: ['GPA', 'courseCode', 'creditHours', 'evenSemester', 'hasSemester', 'isGraduationCase', 'maxCreditHours', 'oddSemester', 'onProbation', 'semesterOffered']


In [5]:
# List all SWRL rules in the ontology
swrl_rules = list(onto.rules())
print(f"Total SWRL Rules: {len(swrl_rules)}")

# Display each rule with its body (antecedent) and head (consequent)
for rule in swrl_rules:
    print("\nRule:")
    print(f"  Name: {rule.name}")

    # Display the antecedent (body) of the rule
    print("  Antecedent (body):")
    for atom in rule.body:
        print(f"    {atom}")

    # Display the consequent (head) of the rule
    print("  Consequent (head):")
    for atom in rule.head:
        print(f"    {atom}")

Total SWRL Rules: 0


In [6]:
# List all classes in the ontology to see their exact names
all_classes = list(onto.classes())
print("Classes in the ontology:")
for cls in all_classes:
    print(cls)

Classes in the ontology:
OntologyNew.Student
OntologyNew.Course


In [7]:
def map_classes_by_label(ontology):
    """
    Creates a dictionary mapping from class labels to the class objects.
    """
    class_label_mapping = {}
    for cls in ontology.classes():
        readable_name = get_readable_name(cls)
        class_label_mapping[readable_name] = cls
    return class_label_mapping


In [8]:
def list_instances_of_class_by_label(ontology, label_to_search):
    """
    Searches for a class by its label and lists all its instances.
    """
    # Get the mapping of class labels to class objects
    class_label_mapping = map_classes_by_label(ontology)

    # Find the class using the provided label
    class_to_search = class_label_mapping.get(label_to_search)

    if class_to_search is None:
        print(f"Class with label '{label_to_search}' not found.")
        return

    # List all instances of the found class
    instances = list(class_to_search.instances())
    if not instances:
        print(f"No instances found for the class with label '{label_to_search}'.")
    else:
        print(f"Instances of the class '{label_to_search}':")
        for instance in instances:
            instance_label = get_readable_name(instance)
            print(f"- {instance}, label: {instance_label}")

In [9]:
# Check instances for the "Course" class
list_instances_of_class_by_label(onto, "Course")

Instances of the class 'Course':
- OntologyNew.Advanced_Computer_Lab2, label: Advanced Computer Lab2
- OntologyNew.Advanced_Computer_Lab3, label: Advanced Computer Lab3
- OntologyNew.Theory_of_Computation, label: Theory of Computation
- OntologyNew.Introduction_to_Computer_Programming, label: Introduction to Computer Programming
- OntologyNew.Artificial_Intelligence, label: Artificial Intelligence
- OntologyNew.Data_Structures_and_Algorithms, label: Data Structures and Algorithms
- OntologyNew.Introduction_to_Computer_Science, label: Introduction to Computer Science
- OntologyNew.Bachelor_Thesis, label: Bachelor Thesis
- OntologyNew.Chem, label: Chem
- OntologyNew.ChemLab, label: ChemLab
- OntologyNew.Compiler, label: Compiler
- OntologyNew.Computer_Graphics, label: Computer Graphics
- OntologyNew.Computer_Organization_and_System_Programming, label: Computer Organization and System Programming
- OntologyNew.Digital_Logic_Design, label: Digital Logic Design
- OntologyNew.Computer_System

In [10]:
# List all individuals in the ontology to check if any exist
all_individuals = list(onto.individuals())
print(f"Total individuals in the ontology: {len(all_individuals)}")
for individual in all_individuals:
    print(individual, individual.is_a)

Total individuals in the ontology: 49
OntologyNew.Advanced_Computer_Lab2 [OntologyNew.Course]
OntologyNew.Advanced_Computer_Lab3 [OntologyNew.Course]
OntologyNew.Theory_of_Computation [OntologyNew.Course]
OntologyNew.Introduction_to_Computer_Programming [OntologyNew.Course]
OntologyNew.Analysis_and_Design_of_Algorithms [owl.Thing]
OntologyNew.Artificial_Intelligence [OntologyNew.Course]
OntologyNew.Data_Structures_and_Algorithms [OntologyNew.Course]
OntologyNew.Introduction_to_Computer_Science [OntologyNew.Course]
OntologyNew.Bachelor_Thesis [OntologyNew.Course]
OntologyNew.Chem [OntologyNew.Course]
OntologyNew.ChemLab [OntologyNew.Course]
OntologyNew.Compiler [OntologyNew.Course]
OntologyNew.Computer_Graphics [OntologyNew.Course]
OntologyNew.Computer_Organization_and_System_Programming [OntologyNew.Course]
OntologyNew.Digital_Logic_Design [OntologyNew.Course]
OntologyNew.Computer_Programming_Lab [owl.Thing]
OntologyNew.Computer_System_Architecture [OntologyNew.Course]
OntologyNew.Comp

In [11]:
# List all classes in the ontology with their IRIs
print("Classes and their IRIs in the ontology:")
for cls in onto.classes():
    print(f"Class: {cls}, IRI: {cls.iri}")

Classes and their IRIs in the ontology:
Class: OntologyNew.Student, IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Student
Class: OntologyNew.Course, IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Course


# Simple Querying

In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("http://localhost:3030/NewOntologyTrial/sparql")
sparql.setQuery("""
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    SELECT DISTINCT ?class ?label
    WHERE { 
        ?class a owl:Class .
        OPTIONAL { ?class rdfs:label ?label }
    }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    class_uri = result["class"]["value"]
    label = result.get("label", {}).get("value", class_uri)  # Fallback to URI if label is missing
    print(label)


Course
Student


In [13]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Configure your SPARQL endpoint
sparql = SPARQLWrapper("http://localhost:3030/NewOntologyTrial/sparql")

def execute_query(query):
    """
    Execute a SPARQL query and return results.
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results

In [14]:
def check_rule_1():
    """
    Rule 1: Invalid Enrollment due to unmet prerequisites.
    """
    query = """
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX main: <http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#>
    SELECT DISTINCT ?s ?course
    WHERE { 
        ?s a main:Student .
        ?s main:takes ?course .
        ?course main: hasPrerequisite ?prereq .
        FILTER NOT EXISTS {
            ?s main:hasCompleted ?prereq .
        }
    }
    """
    return execute_query(query)

In [ ]:
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX main: <http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#>
    SELECT ?course
WHERE {
  ?course a main:Course ;
          main:evenSemester true .
}

# Rule Checking 

In [18]:
from owlready2 import *
import rdflib
def extract_rules(ontology):
    """
    Extract SWRL rules from the ontology.
    """
    return list(ontology.swrl_rules)
extract_rules(onto)

TypeError: 'NoneType' object is not iterable

#   Input parsing and Student instance creation 

In [15]:
class StudentInfoProcessor:
    def __init__(self, ontology):
        self.ontology = ontology

    def find_class_by_label(self, class_label):
        """
        Finds a class in the ontology based on its label.
        """
        for cls in self.ontology.classes():
            if cls.label and class_label in cls.label:
                return cls
        print(f"Class with label '{class_label}' not found.")
        return None

    def find_instance_by_label(self, instance_label):
        """
        Finds an instance in the ontology based on its label.
        """
        for instance in self.ontology.individuals():
            if instance.label and instance_label in instance.label:
                return instance
        print(f"Instance with label '{instance_label}' not found.")
        return None

    def parse_student_info(self, prompt):
        """
        Parses the natural language input prompt to extract student information.
        """
        # Extract GPA
        gpa_match = re.search(r"GPA\s*[:=]\s*(\d+\.\d+)", prompt, re.IGNORECASE)
        gpa = float(gpa_match.group(1)) if gpa_match else 0.0

        # Extract failed courses
        failed_courses_match = re.search(r"Failed Courses\s*[:=]\s*(.+)", prompt, re.IGNORECASE)
        failed_courses = (
            [course.strip() for course in failed_courses_match.group(1).split(",")]
            if failed_courses_match
            else []
        )

        # Extract unattended courses
        unattended_courses_match = re.search(r"Unattended Courses\s*[:=]\s*(.+)", prompt, re.IGNORECASE)
        unattended_courses = (
            [course.strip() for course in unattended_courses_match.group(1).split(",")]
            if unattended_courses_match
            else []
        )

        # Extract graduation case status
        is_graduation_case_match = re.search(r"Graduation Case\s*[:=]\s*(true|false)", prompt, re.IGNORECASE)
        is_graduation_case = is_graduation_case_match.group(1).lower() == "true" if is_graduation_case_match else False

        # Extract the student label
        student_label_match = re.search(r"Student Label\s*[:=]\s*(.+)", prompt, re.IGNORECASE)
        student_label = student_label_match.group(1).strip() if student_label_match else "Student_1"

        return {
            "gpa": gpa,
            "failed_courses": failed_courses,
            "unattended_courses": unattended_courses,
            "is_graduation_case": is_graduation_case,
            "student_label": student_label,
        }

    def create_student_instance(self, student_info):
      student_class = self.find_class_by_label("Student")
      if student_class is None:
          raise ValueError("The 'Student' class could not be found in the ontology.")

      student = student_class(student_info["student_label"])

      # Set GPA and verify
      student.GPA = [student_info["gpa"]]
      print(f"Set GPA for {student_info['student_label']}: {student.GPA}")

      # Set graduation case status and verify
      student.isGraduationCase = [student_info["is_graduation_case"]]
      print(f"Set isGraduationCase for {student_info['student_label']}: {student.isGraduationCase}")

      # Map failed courses
      for course_label in set(student_info["failed_courses"]):
          course_instance = self.find_instance_by_label(course_label)
          if course_instance:
              student.failed.append(course_instance)
              print(f"Mapped failed course {course_label} for {student_info['student_label']}")

      # Map unattended courses
      for course_label in set(student_info["unattended_courses"]):
          course_instance = self.find_instance_by_label(course_label)
          if course_instance:
              student.didNotTakeYet.append(course_instance)
              print(f"Mapped unattended course {course_label} for {student_info['student_label']}")

      # Infer and map completed courses
      all_courses = [ind for ind in self.ontology.individuals() if self.find_class_by_label("Course") in ind.is_a]
      for course in all_courses:
          course_label = course.label.first() if course.label else None
          if course_label and course_label not in student_info["failed_courses"] and course_label not in student_info["unattended_courses"]:
              student.hasCompleted.append(course)
              print(f"Inferred completed course {course_label} for {student_info['student_label']}")

      print(f"Student '{student_info['student_label']}' instance created successfully.")
      return student


In [ ]:
   PREFIX owl: <http://www.w3.org/2002/07/owl#>
   PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
   PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
   PREFIX main: <http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#>
    SELECT ?course ?prerequisite ?creditHours ?oddSemester
WHERE {
  ?course a main:Course ;
          main:creditHours ?creditHours ;
          main:hasPrerequisite ?prerequisite ;
          main:oddSemester ?oddSemester .
  FILTER (?course IN (main:Computer_Graphics, main:Analysis_and_Design_of_Algorithms, 
                      main:Computer_Vision, main:Advanced_Computer_Lab3))
}

In [18]:
# Example prompt
student_prompt = """
GPA: 2.78
Failed Courses: Computer Graphics, Analysis and Design of Algorithms
Unattended Courses: Computer Vision, Advanced Computer Lab3
Graduation Case: true
Student Label: Farida_Test
"""

student_processor = StudentInfoProcessor(onto)

# Parse the student information from the prompt
student_info = student_processor.parse_student_info(student_prompt)

# Create a student instance in the ontology
student_instance = student_processor.create_student_instance(student_info)


Set GPA for Farida_Test: [2.78]
Set isGraduationCase for Farida_Test: [True]
Mapped failed course Analysis and Design of Algorithms for Farida_Test
Mapped failed course Computer Graphics for Farida_Test
Mapped unattended course Advanced Computer Lab3 for Farida_Test
Mapped unattended course Computer Vision for Farida_Test
Inferred completed course Advanced Computer Lab2 for Farida_Test
Inferred completed course Theory of Computation for Farida_Test
Inferred completed course Introduction to Computer Programming for Farida_Test
Inferred completed course Artificial Intelligence for Farida_Test
Inferred completed course Data Structures and Algorithms for Farida_Test
Inferred completed course Introduction to Computer Science for Farida_Test
Inferred completed course Bachelor Thesis for Farida_Test
Inferred completed course Chem for Farida_Test
Inferred completed course ChemLab for Farida_Test
Inferred completed course Compiler for Farida_Test
Inferred completed course Computer Organization 

In [19]:
# Retrieve the instance by IRI pattern (assuming ontology uses "Farida_Test" directly in the IRI)
farida_instance = student_processor.ontology.search_one(iri="*Farida_Test")
if farida_instance:
    print(f"Instance found by IRI: {farida_instance.iri}")
else:
    print("Instance 'Farida_Test' not found by IRI.")

Instance found by IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Farida_Test


In [20]:
# Retrieve the instance by its IRI or existing reference
farida_instance = onto.search_one(iri="*Farida_Test")  # Adjust the pattern if needed

if farida_instance:
    # Assign the label
    farida_instance.label = ["Farida_Test"]
    print(f"Label assigned to Farida_Test: {farida_instance.label}")
else:
    print("Instance 'Farida_Test' could not be found.")

Label assigned to Farida_Test: ['Farida_Test']


In [21]:
# Print all individuals and their labels to confirm presence
print("All instances in the ontology with labels:")
for instance in student_processor.ontology.individuals():
    label = instance.label.first() if instance.label else "No label"
    print(f"Instance IRI: {instance.iri}, Label: {label}")


All instances in the ontology with labels:
Instance IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Advanced_Computer_Lab2, Label: Advanced Computer Lab2
Instance IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Advanced_Computer_Lab3, Label: Advanced Computer Lab3
Instance IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Theory_of_Computation, Label: Theory of Computation
Instance IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Introduction_to_Computer_Programming, Label: Introduction to Computer Programming
Instance IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Analysis_and_Design_of_Algorithms, Label: Analysis and Design of Algorithms
Instance IRI: http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#Artificial_Intelligence, Label: Artificial Intelligence
Instance IRI: http://www.

In [22]:
# Retrieve the 'Farida_Test' instance
farida_instance = student_processor.find_instance_by_label("Farida_Test")

if farida_instance is not None:
    # Check if GPA is assigned and print it
    if hasattr(farida_instance, "GPA"):
        print(f"GPA for Farida_Test: {farida_instance.GPA}")
    else:
        print("GPA property is not assigned to Farida_Test.")
else:
    print("Instance 'Farida_Test' could not be found.")


GPA for Farida_Test: [2.78]


In [23]:
# Retrieve the 'Farida_Test' instance
farida_instance = onto.search_one(iri="*Farida_Test")  # Adjust pattern if necessary

if farida_instance:
    # Check if the instance has a 'failed' property and print related objects
    if hasattr(farida_instance, "failed"):
        failed_courses = farida_instance.failed
        print("Courses related to Farida_Test through the 'failed' property:")
        for course in failed_courses:
            # Print the label if it exists, otherwise print the IRI
            course_label = course.label.first() if course.label else course.iri
            print(f"- {course_label}")
    else:
        print("The 'failed' property is not assigned to Farida_Test.")
else:
    print("Instance 'Farida_Test' could not be found.")


Courses related to Farida_Test through the 'failed' property:
- Analysis and Design of Algorithms
- Computer Graphics
- Analysis and Design of Algorithms
- Computer Graphics


In [24]:
# Retrieve the 'Farida_Test' instance
farida_instance = onto.search_one(iri="*Farida_Test")  # Adjust pattern if necessary

if farida_instance:
    # Check if the instance has a 'didNotTakeYet' property and print related objects
    if hasattr(farida_instance, "didNotTakeYet"):
        unattended_courses = farida_instance.didNotTakeYet
        print("Courses related to Farida_Test through the 'didNotTakeYet' (unattended) property:")
        for course in unattended_courses:
            # Print the label if it exists, otherwise print the IRI
            course_label = course.label.first() if course.label else course.iri
            print(f"- {course_label}")
    else:
        print("The 'didNotTakeYet' (unattended courses) property is not assigned to Farida_Test.")
else:
    print("Instance 'Farida_Test' could not be found.")


Courses related to Farida_Test through the 'didNotTakeYet' (unattended) property:
- Advanced Computer Lab3
- Computer Vision
- Advanced Computer Lab3
- Computer Vision


# In Case we want to remove an instance from the ontology

In [28]:
from owlready2 import destroy_entity

def remove_student_instance(ontology, student_name):
    """
    Removes a student instance from the ontology by name.
    """
    # Search for the student instance by its name
    student_instance = ontology.search_one(iri=f"*{student_name}")

    if student_instance is None:
        print(f"Student instance '{student_name}' not found in the ontology.")
        return

    # Remove the student instance
    destroy_entity(student_instance)
    print(f"Student instance '{student_name}' has been removed from the ontology.")

# Usage example
remove_student_instance(onto, "Farida_Test")

Student instance 'Farida_Test' has been removed from the ontology.


# Save updated Ontology

In [60]:
# Save the updated ontology
onto.save(file="/Users/faridahelmy/Downloads/OntologyNew.rdf", format="rdfxml")  # Save as RDF/XML

# Ollama models

# Example orchestration

In [25]:
student_data = {
    "GPA": 2.78,
    "failed_courses": ["Computer_Graphics", "Analysis_and_Design_of_Algorithms", "Data_Structures_and_Algorithms"],
    "unattended_courses": ["Data_Bases_II", "Advanced_Computer_Lab3", "Theory_of_Computation", "Compiler"],
    "allowed_credit_hours": 34,
}

In [26]:
from rdflib import Graph, Namespace, URIRef

In [48]:
# Define ontology namespace
ONTOLOGY_NS = Namespace("http://www.semanticweb.org/faridahelmy/ontologies/2024/10/untitled-ontology-11#")


In [49]:
# Initialize RDF Graph
g = Graph()
g.parse("/Users/faridahelmy/Downloads/OntologyNew.rdf", format="xml")  # Load your ontology file


<Graph identifier=Naf66379b0bc54e6ca860fd53f677bcbc (<class 'rdflib.graph.Graph'>)>

In [31]:
def get_remaining_courses_details(graph, courses):
    """Fetch details for failed and unattended courses, including semester labels."""
    course_uris = ", ".join([f"<{ONTOLOGY_NS}{course}>" for course in courses])
    query = f"""
    PREFIX : <{ONTOLOGY_NS}>
    SELECT ?course ?prerequisite ?creditHours ?isOddCourse ?isEvenCourse
    WHERE {{
      ?course a :Course ;
              :creditHours ?creditHours .
      OPTIONAL {{ ?course :hasPrerequisite ?prerequisite . }}
      OPTIONAL {{ ?course :oddSemester ?isOddCourse . }}
      OPTIONAL {{ ?course :evenSemester ?isEvenCourse . }}
      FILTER (?course IN ({course_uris}))
    }}
    """
    return graph.query(query)


In [41]:
def get_passed_courses(graph, failed_courses, unattended_courses):
    """
    Infer passed courses dynamically by excluding failed and unattended courses.
    """
    # Convert lists to SPARQL-compatible format
    failed_courses_uris = ", ".join([f"<{ONTOLOGY_NS}{course}>" for course in failed_courses])
    unattended_courses_uris = ", ".join([f"<{ONTOLOGY_NS}{course}>" for course in unattended_courses])

    # SPARQL query to fetch all courses not in failed or unattended courses
    query = f"""
    PREFIX : <{ONTOLOGY_NS}>
    SELECT ?course
    WHERE {{
      ?course a :Course .
      FILTER (?course NOT IN ({failed_courses_uris}, {unattended_courses_uris}))
    }}
    """
    results = graph.query(query)

    # Collect all passed courses
    passed_courses = [str(row[0]).split("#")[-1] for row in results]

    return passed_courses


In [32]:
def get_course_prerequisites(graph, course):
    """Fetch prerequisites for a specific course."""
    query = f"""
    PREFIX : <{ONTOLOGY_NS}>
    SELECT ?prerequisite
    WHERE {{
      <{ONTOLOGY_NS}{course}> :hasPrerequisite ?prerequisite .
    }}
    """
    return [str(row[0]).split("#")[-1] for row in graph.query(query)]

In [33]:
def split_courses_by_semester(course_details):
    """Split courses into odd and even semesters."""
    odd_semester = [course for course, details in course_details.items() if details["oddSemester"]]
    even_semester = [course for course, details in course_details.items() if not details["oddSemester"]]
    return odd_semester, even_semester

In [34]:
def prioritize_courses(failed_courses, unattended_courses, odd_courses, even_courses):
    """Prioritize failed courses over unattended courses."""
    prioritized_odd = [c for c in failed_courses if c in odd_courses] + [c for c in unattended_courses if c in odd_courses]
    prioritized_even = [c for c in failed_courses if c in even_courses] + [c for c in unattended_courses if c in even_courses]
    return prioritized_odd, prioritized_even

In [35]:
def schedule_courses(odd_courses, even_courses, credit_limit, course_details):
    """Schedule courses without exceeding the credit hour limit."""
    schedule = {"Odd Semester": [], "Even Semester": []}
    current_credits = 0

    for semester, courses in [("Odd Semester", odd_courses), ("Even Semester", even_courses)]:
        for course in courses:
            if current_credits + course_details[course]["creditHours"] <= credit_limit:
                schedule[semester].append(course)
                current_credits += course_details[course]["creditHours"]
    return schedule

In [45]:
def generate_graduation_plan(graph, student_data):
    # Step 1: Fetch course details
    print("Fetching course details...")
    all_courses = student_data["failed_courses"] + student_data["unattended_courses"]
    print("All Courses:", all_courses)
    results = get_remaining_courses_details(graph, all_courses)
    
    

    # Step 2: Process course details
    print("Processing course details...")
    course_details = {}
    for row in results:
        course = str(row[0]).split("#")[-1]
        prerequisite = str(row[1]).split("#")[-1] if row[1] else None
        credit_hours = float(row[2].toPython())  # Handles decimal credit hours
        odd_semester = row[3].toPython() if row[3] else False  # Default to False if missing
        even_semester = row[4].toPython() if row[4] else False  # Default to False if missing

        if course not in course_details:
            course_details[course] = {
                "prerequisites": [],
                "creditHours": credit_hours,
                "oddSemester": odd_semester,
                "evenSemester": even_semester,
            }
        if prerequisite:
            course_details[course]["prerequisites"].append(prerequisite)

    # Log course details
    print("Course Details:", course_details)

    # Step 3: Infer passed courses dynamically
    print("Inferring passed courses...")
    student_data["passed_courses"] = get_passed_courses(
        graph=graph,
        failed_courses=student_data["failed_courses"],
        unattended_courses=student_data["unattended_courses"],
    )
    print("Passed Courses:", student_data["passed_courses"])

    # Step 4: Filter courses based on prerequisites
    print("Filtering eligible courses...")
    eligible_courses = {
    course: details
    for course, details in course_details.items()
    if (not details.get("prerequisites", []) or  # Include courses with no prerequisites or an empty list
        all(prereq in student_data["passed_courses"] for prereq in details["prerequisites"]))
}
    print("Eligible Courses:", eligible_courses)

    # Step 5: Split courses into odd/even semesters
    print("Splitting courses by semester...")
    odd_courses, even_courses = split_courses_by_semester(eligible_courses)
    print("Odd Semester Courses:", odd_courses)
    print("Even Semester Courses:", even_courses)

    # Step 6: Prioritize courses
    print("Prioritizing courses...")
    prioritized_odd, prioritized_even = prioritize_courses(
        student_data["failed_courses"],
        student_data["unattended_courses"],
        odd_courses,
        even_courses,
    )
    print("Prioritized Odd Semester:", prioritized_odd)
    print("Prioritized Even Semester:", prioritized_even)

    # Step 7: Schedule courses
    print("Scheduling courses...")
    schedule = schedule_courses(
        prioritized_odd, prioritized_even, student_data["allowed_credit_hours"], eligible_courses
    )
    print("Final Schedule:", schedule)

    return schedule


In [51]:
# Generate Graduation Plan
graduation_plan = generate_graduation_plan(g, student_data)

# Output Plan
print("Graduation Plan:", graduation_plan)

Fetching course details...
All Courses: ['Computer_Graphics', 'Analysis_and_Design_of_Algorithms', 'Data_Structures_and_Algorithms', 'Data_Bases_II', 'Advanced_Computer_Lab3', 'Theory_of_Computation', 'Compiler']
Processing course details...
Course Details: {'Advanced_Computer_Lab3': {'prerequisites': ['Theory_of_Computation'], 'creditHours': 4.0, 'oddSemester': False, 'evenSemester': True}, 'Analysis_and_Design_of_Algorithms': {'prerequisites': [], 'creditHours': 4.0, 'oddSemester': True, 'evenSemester': False}, 'Compiler': {'prerequisites': ['Theory_of_Computation'], 'creditHours': 4.0, 'oddSemester': False, 'evenSemester': True}, 'Computer_Graphics': {'prerequisites': ['Data_Structures_and_Algorithms', 'Introduction_to_Computer_Programming'], 'creditHours': 6.0, 'oddSemester': True, 'evenSemester': False}, 'Data_Bases_II': {'prerequisites': ['Data_Base_I'], 'creditHours': 4.0, 'oddSemester': False, 'evenSemester': True}, 'Data_Structures_and_Algorithms': {'prerequisites': ['Introduc